# Seismic CropBatch tutorial

Welcome! This notebook shows how to use `SeismicCropBatch` to load data from cubes, create segmentation masks from labels, and more!

In [ ]:
import os
import sys
import warnings

import numpy as np
import matplotlib.pyplot as plt

sys.path.append('..')
from seismiqb.batchflow import FilesIndex
from seismiqb.batchflow import B, V, C, L, F, D, P, R
from seismiqb import SeismicCropBatch, SeismicGeometry, SeismicCubeset

from glob import glob

#### First of all, we create `SeismicCubeset`

In [ ]:
path_data_0 = '/notebooks/SEISMIC_DATA/CUBE_1/E_anon.hdf5'
path_data_1 = '/notebooks/SEISMIC_DATA/CUBE_3/P_cube.hdf5'
path_data_2 = '/notebooks/SEISMIC_DATA/CUBE_VUONGMK/Repaired_cube.hdf5'

dsi = FilesIndex(path=[path_data_0, path_data_1, path_data_2], no_ext=True)
ds = SeismicCubeset(dsi)

ds = (ds.load_geometries()
        .load_point_clouds(path = path_pc_saved)
        .load_labels()
        .load_samplers(p=[0.4, 0.2, 0.4])
      )

# ~80 seconds

#### Create batch

* First action, `load_component`, creates references to geometries and labels in instance of `SeismicCropBatch`. That is done for convinience and ease of access of these entities.

* Then we use `crop`. It creates positions to cut crops of desired shape from cubes. Note that it does not load anything from disk: only positions are created.

* Heavy lifting of disk I/O is performed by `load_cubes`, that takes positions (created by `crop`) and actually gets data from cube.

* `load_masks` creates segmentation mask of the same size as cutted crop from `labels` dictionary.

In [ ]:
demo_batch = (ds.p.load_component(src=[D('geometries'), D('labels')],
                                  dst=['geometries', 'labels'])
                  .crop(points=L(ds.sampler.sample, 16), shape=[3, 512, 512])
                  .load_cubes(dst='data_crops')
                  .load_masks(dst='mask_crops')
              )

#### Check contents of `data_crops` and `mask_crops` components of batch:

In [ ]:
for cube in np.random.choice(16, 2):
    print(demo_batch.indices[cube][:-10])
    iline = 0
    
    img = demo_batch.data_crops[cube][iline, :, :].T
    mask = demo_batch.mask_crops[cube][iline, :, :].T
    
    fig, ax = plt.subplots(1, 2, figsize=(25, 10))

    ax[0].imshow(img, cmap='gray')
    ax[0].set_title('Cube slice')
    
    ax[1].imshow(masks, cmap="Blues")
    ax[1].imshow(img, cmap="gray", alpha=0.5)
    ax[1].imshow(masks, cmap="Blues", alpha=0.1)
    ax[1].set_title('Mask')
   
    plt.show()

#### Lets add some data augmentations:
* `rotate_axis`. 